In [9]:
# Data structures for users and options
users = {}    # user_id: {'holdings': {option_id: quantity}}
options = {}  # option_id: {'price': float, 'total_supply': int}

# Function to get options prices and IDs
def get_options():
    return {option_id: options[option_id]['price'] for option_id in options}

# Function to create a new option
def create_option(option_id, initial_price, total_supply):
    if option_id in options:
        print(f"Option {option_id} already exists.")
    else:
        options[option_id] = {'price': initial_price, 'total_supply': total_supply}
        print(f"Option {option_id} created with price {initial_price} and total supply {total_supply}.")

# Function for a user to buy an option
def buy_option(user_id, option_id, amount):
    if option_id not in options:
        print(f"Option {option_id} does not exist.")
        return
    option = options[option_id]
    if amount > option['total_supply']:
        print(f"Not enough supply of option {option_id} to buy {amount} units.")
        return
    # Update user's holdings
    if user_id not in users:
        users[user_id] = {'holdings': {}}
    user = users[user_id]
    holdings = user['holdings']
    holdings[option_id] = holdings.get(option_id, 0) + amount
    # Decrease total supply
    option['total_supply'] -= amount
    # Adjust price based on simple supply and demand
    option['price'] *= 1 + (amount / (option['total_supply'] + amount)) * 0.01
    print(f"User {user_id} bought {amount} units of option {option_id}.")

# Function for a user to sell an option
def sell_option(user_id, option_id, amount):
    if option_id not in options:
        print(f"Option {option_id} does not exist.")
        return
    if user_id not in users or option_id not in users[user_id]['holdings']:
        print(f"User {user_id} does not hold option {option_id}.")
        return
    user = users[user_id]
    holdings = user['holdings']
    if holdings[option_id] < amount:
        print(f"User {user_id} does not have enough of option {option_id} to sell.")
        return
    # Update user's holdings
    holdings[option_id] -= amount
    # Increase total supply
    option = options[option_id]
    option['total_supply'] += amount
    # Adjust price based on simple supply and demand
    option['price'] *= 1 - (amount / option['total_supply']) * 0.01
    print(f"User {user_id} sold {amount} units of option {option_id}.")

# Example usage
if __name__ == "__main__":
    # Create a new option
    create_option('opt1', 100.0, 1000)
    # Display current options
    print("Current Options:", get_options())
    # User buys the option
    buy_option('user1', 'opt1', 100)
    # Display updated options
    print("Options after purchase:", get_options())
    # User sells some of the option
    sell_option('user1', 'opt1', 50)
    # Display updated options
    print("Options after sale:", get_options())
    # Display user holdings
    print("User Holdings:", users['user1']['holdings'])


Option opt1 created with price 100.0 and total supply 1000.
Current Options: {'opt1': 100.0}
User user1 bought 100 units of option opt1.
Options after purchase: {'opt1': 100.1}
User user1 sold 50 units of option opt1.
Options after sale: {'opt1': 100.04731578947367}
User Holdings: {'opt1': 50}


In [10]:
import math

# Data structures for users and options
users = {}    # user_id: {'holdings': {option_id: quantity}}
options = {}  # option_id: {'S': float, 'K': float, 'T': float, 'r': float, 'sigma': float, 'type': 'call'/'put', 'price': float}

# Approximate cumulative distribution function for the standard normal distribution
def cnd(x):
    """Cumulative standard normal distribution function approximation."""
    # Constants in the approximation
    a1 = 0.319381530
    a2 = -0.356563782
    a3 = 1.781477937
    a4 = -1.821255978
    a5 = 1.330274429
    p  = 0.2316419

    L = abs(x)
    t = 1.0 / (1.0 + p * L)
    poly = t * (a1 + t * (a2 + t * (a3 + t * (a4 + t * a5))))
    value = 1.0 - (1.0 / math.sqrt(2 * math.pi)) * math.exp(-L * L / 2.0) * poly
    if x < 0:
        value = 1.0 - value
    return value

# Black-Scholes formula implementation without external libraries
def black_scholes(S, K, T, r, sigma, option_type):
    """Calculate the Black-Scholes price for a call or put option."""
    if T <= 0:
        # Option has expired
        if option_type == 'call':
            return max(0.0, S - K)
        elif option_type == 'put':
            return max(0.0, K - S)
    else:
        d1 = (math.log(S / K) + (r + 0.5 * sigma * sigma) * T) / (sigma * math.sqrt(T))
        d2 = d1 - sigma * math.sqrt(T)
        if option_type == 'call':
            price = S * cnd(d1) - K * math.exp(-r * T) * cnd(d2)
        elif option_type == 'put':
            price = K * math.exp(-r * T) * cnd(-d2) - S * cnd(-d1)
        else:
            raise ValueError("option_type must be 'call' or 'put'")
        return price

# Function to get options prices and IDs
def get_options():
    return {option_id: options[option_id]['price'] for option_id in options}

# Function to create a new option
def create_option(option_id, S, K, T, r, sigma, option_type):
    if option_id in options:
        print(f"Option {option_id} already exists.")
    else:
        price = black_scholes(S, K, T, r, sigma, option_type)
        options[option_id] = {
            'S': S,
            'K': K,
            'T': T,
            'r': r,
            'sigma': sigma,
            'type': option_type,
            'price': price
        }
        print(f"Option {option_id} created with price {price:.2f}.")

# Function to update option prices (simulate market fluctuations)
def update_market(S_new, days_passed=1):
    """Update the underlying stock price and recalculate option prices."""
    for option_id, option in options.items():
        option['S'] = S_new
        # Decrease time to expiration
        T_new = option['T'] - days_passed / 252  # Assuming 252 trading days in a year
        option['T'] = max(T_new, 0.0)  # Time cannot be negative
        option['price'] = black_scholes(option['S'], option['K'], option['T'], option['r'], option['sigma'], option['type'])

# Function for a user to buy an option
def buy_option(user_id, option_id, amount):
    if option_id not in options:
        print(f"Option {option_id} does not exist.")
        return
    # Update user's holdings
    if user_id not in users:
        users[user_id] = {'holdings': {}}
    user = users[user_id]
    holdings = user['holdings']
    holdings[option_id] = holdings.get(option_id, 0) + amount
    print(f"User {user_id} bought {amount} units of option {option_id} at price {options[option_id]['price']:.2f} each.")

# Function for a user to sell an option
def sell_option(user_id, option_id, amount):
    if option_id not in options:
        print(f"Option {option_id} does not exist.")
        return
    if user_id not in users or option_id not in users[user_id]['holdings']:
        print(f"User {user_id} does not hold option {option_id}.")
        return
    user = users[user_id]
    holdings = user['holdings']
    if holdings[option_id] < amount:
        print(f"User {user_id} does not have enough of option {option_id} to sell.")
        return
    holdings[option_id] -= amount
    print(f"User {user_id} sold {amount} units of option {option_id} at price {options[option_id]['price']:.2f} each.")

# Function to display market fluctuations (pricing for all options)
def display_market():
    print("Current Market Prices:")
    for option_id, option in options.items():
        print(f"Option {option_id}: Price = {option['price']:.2f}, Underlying Price = {option['S']:.2f}, Time to Expiration = {option['T']:.4f}")

# Simulate the market and record option prices over time
def simulate_market(option_id, S_values, days):
    """Simulate the market and collect option prices over a number of days."""
    prices = []
    for day in range(days):
        S_new = S_values[day]
        update_market(S_new, days_passed=1)
        option_price = options[option_id]['price']
        prices.append(option_price)
    return prices

# Example usage
if __name__ == "__main__":
    # Create a new option
    create_option('opt1', S=100.0, K=100.0, T=1.0, r=0.05, sigma=0.2, option_type='call')
    # Display current options
    display_market()
    # User buys the option
    buy_option('user1', 'opt1', 10)

    # Simulate market fluctuations over 100 days
    days = 100
    S_values = [100 + 0.5 * day for day in range(days)]  # Simulate underlying price increasing over time
    prices = simulate_market('opt1', S_values, days)

    # Output the simulated option prices
    print("\nSimulated Option Prices over Time:")
    print("Day\tUnderlying Price\tOption Price")
    for day in range(days):
        print(f"{day+1}\t{S_values[day]:.2f}\t\t\t{prices[day]:.2f}")

    # User sells some of the option after simulation
    sell_option('user1', 'opt1', 5)
    # Display user holdings
    print("User Holdings:", users['user1']['holdings'])


Option opt1 created with price 10.45.
Current Market Prices:
Option opt1: Price = 10.45, Underlying Price = 100.00, Time to Expiration = 1.0000
User user1 bought 10 units of option opt1 at price 10.45 each.

Simulated Option Prices over Time:
Day	Underlying Price	Option Price
1	100.00			10.43
2	100.50			10.72
3	101.00			11.02
4	101.50			11.32
5	102.00			11.63
6	102.50			11.94
7	103.00			12.26
8	103.50			12.58
9	104.00			12.91
10	104.50			13.23
11	105.00			13.57
12	105.50			13.90
13	106.00			14.24
14	106.50			14.59
15	107.00			14.94
16	107.50			15.29
17	108.00			15.65
18	108.50			16.01
19	109.00			16.37
20	109.50			16.74
21	110.00			17.11
22	110.50			17.48
23	111.00			17.86
24	111.50			18.24
25	112.00			18.62
26	112.50			19.01
27	113.00			19.40
28	113.50			19.79
29	114.00			20.18
30	114.50			20.58
31	115.00			20.98
32	115.50			21.39
33	116.00			21.80
34	116.50			22.21
35	117.00			22.62
36	117.50			23.03
37	118.00			23.45
38	118.50			23.87
39	119.00			24.29
40	119.50			24.72
41	120.00			

In [ ]:
# (define-data-var next-option-id uint u0)

# (define-map options uint {quantity: uint, price: uint})

# (define-map holdings {user: principal, option-id: uint} uint)


# ;; Define price adjustment factor (e.g., 10% per unit change)
# (define-constant PRICE_ADJUSTMENT u10)

# ;; Helper functions
# (define-private (get-list-length (lst (list 20 (tuple (quantity uint) (price uint)))))
#   (ok (len lst))
# )
# (define-read-only (get-option (option-id uint))
#   (map-get? options option-id)
# )
# (define-read-only (get-holdings (user principal) (option-id uint))
#   (map-get? holdings {user: user, option-id: option-id})
# )


# ;; Create option function with initial price
# (define-public (create-option (quantity uint) (initial-price uint))
#   (let ((option-id (var-get next-option-id)))
#     (begin
#       (map-set options option-id {quantity: quantity, price: initial-price})
#       (var-set next-option-id (+ option-id u1))
#       (ok option-id) ;; Returns the option-id
#     )
#   )
# )


# ;; Adjust the price of an option
# (define-private (adjust-price (current-price uint) (quantity uint) (is-buy bool))
#   (if is-buy
#       (+ current-price (* quantity PRICE_ADJUSTMENT))
#       (- current-price (* quantity PRICE_ADJUSTMENT))
#   )
# )


# ;; Buy option function
# (define-public (buy-option (option-id uint) (quantity uint))
#   (let ((option-data (map-get? options option-id)))
#     (if (is-some option-data)
#         (let (
#               (option (unwrap-panic option-data))
#               (current-quantity (get quantity option))
#               (current-price (get price option))
#               (new-quantity (+ current-quantity quantity))
#               (new-price (adjust-price current-price quantity true))
#               (user-holdings (default-to u0 (map-get? holdings {user: tx-sender, option-id: option-id})))
#               (new-holdings (+ user-holdings quantity))
#              )
#           (begin
#             ;; Update user's holdings
#             (map-set holdings {user: tx-sender, option-id: option-id} new-holdings)
#             ;; Update option quantity and price
#             (map-set options option-id {quantity: new-quantity, price: new-price})
#             (ok new-holdings)
#           )
#         )
#         (err u2) ;; Error if option does not exist
#     )
#   )
# )


# ;; Sell option function
# (define-public (sell-option (option-id uint) (quantity uint))
#   (let ((option-data (map-get? options option-id)))
#     (if (is-some option-data)
#         (let (
#               (option (unwrap-panic option-data))
#               (current-quantity (get quantity option))
#               (current-price (get price option))
#               (user-holdings (default-to u0 (map-get? holdings {user: tx-sender, option-id: option-id})))
#              )
#           (if (>= user-holdings quantity)
#               (let (
#                     (new-holdings (- user-holdings quantity))
#                     (new-quantity (- current-quantity quantity))
#                     (new-price (adjust-price current-price quantity false))
#                    )
#                 (begin
#                   ;; Update user's holdings
#                   (map-set holdings {user: tx-sender, option-id: option-id} new-holdings)
#                   ;; Update option quantity and price
#                   (map-set options option-id {quantity: new-quantity, price: new-price})
#                   (ok new-holdings)
#                 )
#               )
#               (err u3) ;; Error if insufficient holdings
#           )
#         )
#         (err u2) ;; Error if option does not exist
#     )
#   )
# )
